In [1]:
import json
import pandas as pd
from shapely.ops import cascaded_union
from pandas.io.json import json_normalize
from shapely.geometry import Point, Polygon
import geopandas as gpd
from folium import Map, Marker, GeoJson, features
from h3 import h3
import requests
import zipfile
import xmltodict, io

In [2]:
import time
ts = round(time.time())
print(ts)

1587399883


In [3]:
resp = requests.get("http://imrad.sire.gov.co:8080/radar/reflectividad.kmz?rand=%s" % ts)
k = zipfile.ZipFile(io.BytesIO(resp.content))
k.extractall('reflec')
km = k.open('doc.kml', 'r').read()
o = xmltodict.parse(km)
img_radar = o["kml"]["Folder"]["GroundOverlay"]["Icon"]["href"]
o

OrderedDict([('kml',
              OrderedDict([('@xmlns', 'http://earth.google.com/kml/2.2'),
                           ('Folder',
                            OrderedDict([('name',
                                          '1399BOG-Intensity (Horizontal)-Mon Apr 20 16:13:48 2020 GMT'),
                                         ('ExtendedData',
                                          OrderedDict([('@xmlns:edge',
                                                        'http://eecradar.com'),
                                                       ('edge:Moment',
                                                        'Intensity (Horizontal)'),
                                                       ('edge:MomentAbbreviation',
                                                        'ZH')])),
                                         ('lookAt',
                                          OrderedDict([('longitude',
                                                        '-74.11397552490234'),

In [4]:
from PIL import Image 
Image.open('reflec/%s' % img_radar).show()
iconmm = features.CustomIcon('reflec/%s' % img_radar, icon_size=(500,500))

In [5]:
## Just use hexagon resolution 7 to be able to get weather data 
with open('source/map_bogota_hex8.json') as f:
    bogota_hex7 = json.load(f)
    
mapa_bogota = Map(location= [4.626853707360377,-74.0682993486343], zoom_start=11, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
Marker([4.626853707360377,-74.0682993486343], icon=iconmm).add_to(mapa_bogota)

GeoJson(
        bogota_hex7,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'blue',
            'weight': 1,
            'fillOpacity': 0
        }, 
        name = "Hexagons_b" 
    ).add_to(mapa_bogota)

mapa_bogota

### Get Lat and long per pixel from Layer

In [50]:
img_loc = o["kml"]["Folder"]["GroundOverlay"]["LatLonBox"]
hor_loc = float(img_loc["east"]) - float(img_loc["west"])
ver_loc = float(img_loc["north"]) - float(img_loc["south"])
print("Distance horizontal " + str(hor_loc) + " and vertical " + str(ver_loc) + "layer")

Distance horizontal 1.0815526954502985 and vertical 1.085170655991746layer


In [51]:
hor_pix = hor_loc/120
ver_pix = ver_loc/120
print("horisontal and vertical per pixel " + str(hor_pix) + ", " + str(ver_pix))

horisontal and vertical per pixel 0.009012939128752487, 0.009043088799931216


In [52]:
bogota_hex = json.loads(bogota_hex7)
bogota_hex =  gpd.GeoDataFrame(bogota_hex['features'])
bogota_hex.head(2)

,type,id,geometry,properties
0,Feature,8866e429a5fffff,"{'type': 'Polygon', 'coordinates': [[[-74.1645...",{'value': 0}
1,Feature,8866e091d9fffff,"{'type': 'Polygon', 'coordinates': [[[-74.1249...",{'value': 0}


In [53]:
bogota_hex['centroid'] = bogota_hex['id'].map(lambda x: h3.h3_to_geo(x))
bogota_hex.head(2)

,type,id,geometry,properties,centroid
0,Feature,8866e429a5fffff,"{'type': 'Polygon', 'coordinates': [[[-74.1645...",{'value': 0},"[4.694945057874073, -74.16403674174285]"
1,Feature,8866e091d9fffff,"{'type': 'Polygon', 'coordinates': [[[-74.1249...",{'value': 0},"[4.565469127518143, -74.124410600837]"


### Map Centroids with Image pixels

In [54]:
img_x_axis = bogota_hex['centroid'][0][1] - float(img_loc["west"])
print(img_x_axis)
print(img_x_axis/hor_pix)

0.49071513088463803
54.44562798823203


In [55]:
img_y_axis = float(img_loc["north"]) - bogota_hex['centroid'][0][0] 
print(img_y_axis)
print(round(img_y_axis/ver_pix))

0.522805069738089
58


In [56]:
#get pixel x, y coordinates from a geolocation list
def getPixels_yx(_list):
    img_x_axis = _list[1] - float(img_loc["west"])
    img_y_axis = float(img_loc["north"]) - _list[0] 
    return [round(img_y_axis/hor_pix), round(img_x_axis/ver_pix)]

In [57]:
bogota_hex['pixel_coord'] = bogota_hex['centroid'].map(lambda x: getPixels_yx(x))

In [58]:
bogota_hex.to_csv('source/map_bogota_hex8pix.csv')